В этом проекте рассматриваются данные о государственных закупках Российской Федерации, проведённых в формате открытых электронных аукционов. Каждый элемент набора данных представляет собой отдельный аукцион с характеристиками: начальная цена, количество участников, параметры торгов и другие особенности.

Цель проекта — спрогнозировать изменение цены (PriceDynamicsString), отражающее, насколько стоимость контракта снизилась в ходе торгов по сравнению с изначальной стартовой ценой.

Для решения задачи применяется модель линейной регрессии, обученная на очищенных и предобработанных данных.

In [114]:
import pandas as pd
df=pd.read_excel('Procurements_students.xlsx')
df

,Unnamed: 0,LotPrice,StartingPrice,PriceDynamicsString,BidCount,AuctionDurationSec,WithStep,StepType,IsRaise,CanImproveBid,...,MinStep,LotName,AuctionStartDate,sleep,number,contract_time,msp,organizator,obesp_zayavki,place
0,0,351555.48,351555.48,"0,500001",1,1220,True,Процент от НМЦ,False,True,...,1757.78,Оказание услуг по осуществлению строительного ...,14.11.2019 07:15:00 (по московскому времени),1,2,NaN,Не установлены,"СНОФ ""РО КАПИТАЛЬНОГО РЕМОНТА МКД ЧЕЛЯБИНСКОЙ ...","3 515,55 (Российский рубль)","454048,Челябинская область,г. Челябинск,ул. О..."
1,1,53600.00,53600.00,"0,500000",1,916,True,Процент от НМЦ,False,False,...,268.00,Поставка молока коровьего ультра пастеризованного,05.03.2018 10:00:00 (по московскому времени),1,2,от 10 до 20 календарных дней,Не установлены,АО 'ЛУК',"0,00 (Российский рубль)","140011, Московская область, город Люберцы, ули..."
2,2,410000.00,410000.00,"34,146341",32,2309,True,Процент от НМЦ,False,False,...,2050.00,Поставка шкафов,14.11.2017 08:30:00 (по московскому времени),1,7,от 10 до 20 календарных дней,Не установлены,Копейский реабилитационный центр для лиц с умс...,"0,00 (Российский рубль)","456654,Челябинская область,город Копейск,Елец..."
3,3,1465502.89,1465502.89,0.999999,2,925,True,Процент от НМЦ,False,False,...,7327.51,на право заключения договора на поставку мебели,23.03.2017 09:00:00 (по московскому времени),0,2,NaN,Не установлены,"ФГБУ ""НМИЦ АГП ИМ. В.И.КУЛАКОВА"" МИНЗДРАВА РОССИИ","14 655,03 (Российский рубль)","117997, город Москва, ул. Академика Опарина, д. 4"
4,4,445524.00,445524.00,11.166990,21,1137,True,Процент от НМЦ,False,True,...,2227.62,Оказание услуг по изготовлению полиграфической...,11.12.2017 11:00:00 (по московскому времени),0,2,от 10 до 20 календарных дней,Не установлены,"ГАУК МО ""МОГТЮЗ""","4 455,24 (Российский рубль)","115516,город Москва,Прохладная улица, 28meta ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,9425000.00,9425000.00,"10,672414",21,4049,True,Процент от НМЦ,False,False,...,47125.00,сульфат аммония,29.08.2019 10:00:00 (по московскому времени),0,2,от 10 до 20 календарных дней,Не установлены,ФГУП «РАССВЕТ-СТАВРОПОЛЬЕ»,"0,00 (Российский рубль)","356350,Ставропольский край,Новоселицкий р-н, ..."
996,996,26368538.00,26368538.00,"39,000000",71,4230,True,Процент от НМЦ,False,True,...,131842.69,Выполнение работ по ремонту или замене лифтово...,12.03.2018 12:00:00 (по московскому времени),7,16,NaN,Не установлены,"СНОФ ""РО КАПИТАЛЬНОГО РЕМОНТА МКД ЧЕЛЯБИНСКОЙ ...","1 318 426,90 (Российский рубль)","454048,Челябинская область,г. Челябинск,ул. О..."
997,997,12851.58,12851.58,"0,500016",1,334,True,Процент от НМЦ,False,False,...,64.26,Поставка товара: Датчики пульсоксиметрические ...,25.09.2017 09:00:00 (по московскому времени),1,2,от 10 до 20 календарных дней,Не установлены,"ГАУЗ СО ""ЦГБ № 20""","128,52 (Российский рубль)","620010,Свердловская область,город Екатеринбур..."
998,998,895000.00,895000.00,"23,500000",47,3728,True,Процент от НМЦ,False,False,...,4475.00,Закупка бумаги для нужд ФБУН ЦНИИ Эпидемиологи...,19.03.2018 09:00:00 (по московскому времени),0,2,от 10 до 20 календарных дней,Торги только для субъектов МСП,ФБУН ЦНИИ ЭПИДЕМИОЛОГИИ РОСПОТРЕБНАДЗОРА,"17 900,00 (Российский рубль)","111123,город Москва,Новогиреевская ул., д.3аm..."


In [115]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

def preprocess(data):
    df = data.copy()

    # Удаление ненужного индекса
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)

    # Преобразование целевой переменной (PriceDynamicsString) в числовой формат
    df['PriceDynamicsString'] = df['PriceDynamicsString'].replace('Не определено', np.nan)
    df['PriceDynamicsString'] = df['PriceDynamicsString'].str.replace(',', '.').astype(float)

    # Удаление ненужных текстовых столбцов
    drop_columns = ['LotName', 'organizator', 'place', 'contract_time', 'obesp_zayavki', 'AuctionStartDate', 'IsRaise', 'StepType', 'msp', 'WithStep', 'CanImproveBid', 'MinStep', 'MaxStep']
    df.drop(columns=[col for col in drop_columns if col in df.columns], inplace=True)

    # Преобразование булевых переменных в 0 и 1
    #bool_cols = ['WithStep', 'CanImproveBid']
    #df[bool_cols] = df[bool_cols].astype(int)

    df = df.dropna()

    return df

preprocess(df)

,LotPrice,StartingPrice,PriceDynamicsString,BidCount,AuctionDurationSec,WithSecondPhase,TradeType,sleep,number
0,351555.48,351555.48,0.500001,1,1220,True,1,1,2
1,53600.00,53600.00,0.500000,1,916,False,1,1,2
2,410000.00,410000.00,34.146341,32,2309,False,1,1,7
3,1465502.89,1465502.89,0.999999,2,925,False,1,0,2
4,445524.00,445524.00,11.166990,21,1137,False,1,0,2
...,...,...,...,...,...,...,...,...,...
995,9425000.00,9425000.00,10.672414,21,4049,False,1,0,2
996,26368538.00,26368538.00,39.000000,71,4230,True,1,7,16
997,12851.58,12851.58,0.500016,1,334,False,1,1,2
998,895000.00,895000.00,23.500000,47,3728,False,1,0,2


In [116]:

df=pd.read_excel('Procurements_students.xlsx')

df_processed = preprocess(df)

# Разделение данных на X и y
X = df_processed.drop(columns=['PriceDynamicsString']) 
y = df_processed['PriceDynamicsString'] 

# Разделение данных на обучающую и тестовую выборки (80% - train, 20% - test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Оценка модели
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.2f}")


R²: 0.6668
MAE: 7.34


In [117]:
# Вычисление RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

11.271986639260474